In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Arlit,NE,2021-01-15 03:07:44,18.7369,7.3853,65.86,26,0,6.15
1,1,Puerto Ayora,EC,2021-01-15 03:09:47,-0.7393,-90.3518,75.00,94,93,7.00
2,2,Ushuaia,AR,2021-01-15 03:11:20,-54.8000,-68.3000,46.40,71,75,11.50
3,3,Talnakh,RU,2021-01-15 03:09:57,69.4865,88.3972,-40.31,77,19,2.82
4,4,Bluff,NZ,2021-01-15 03:11:31,-46.6000,168.3333,73.00,63,7,8.99


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [9]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [16]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Puerto Ayora,EC,2021-01-15 03:09:47,-0.7393,-90.3518,75.00,94,93,7.00
6,6,Rikitea,PF,2021-01-15 03:11:31,-23.1203,-134.9692,78.22,71,62,9.04
21,21,Hithadhoo,MV,2021-01-15 03:11:33,-0.6000,73.0833,81.90,78,83,17.92
23,23,Saint-Joseph,RE,2021-01-15 03:11:33,-21.3667,55.6167,79.00,73,40,4.61
26,26,Buala,SB,2021-01-15 03:10:20,-8.1450,159.5921,86.09,71,44,8.70
30,30,Kapaa,US,2021-01-15 03:09:36,22.0752,-159.3190,78.80,73,1,9.22
33,33,Hilo,US,2021-01-15 03:11:34,19.7297,-155.0900,78.80,65,20,13.80
44,44,Arraial Do Cabo,BR,2021-01-15 03:09:28,-22.9661,-42.0278,75.40,90,17,16.37
46,46,Inhambane,MZ,2021-01-15 03:11:35,-23.8650,35.3833,78.67,87,30,7.63
50,50,Atuona,PF,2021-01-15 03:09:36,-9.8000,-139.0333,77.65,74,0,11.12


In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.00,-0.7393,-90.3518,
6,Rikitea,PF,78.22,-23.1203,-134.9692,
21,Hithadhoo,MV,81.90,-0.6000,73.0833,
23,Saint-Joseph,RE,79.00,-21.3667,55.6167,
26,Buala,SB,86.09,-8.1450,159.5921,
30,Kapaa,US,78.80,22.0752,-159.3190,
33,Hilo,US,78.80,19.7297,-155.0900,
44,Arraial Do Cabo,BR,75.40,-22.9661,-42.0278,
46,Inhambane,MZ,78.67,-23.8650,35.3833,
50,Atuona,PF,77.65,-9.8000,-139.0333,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [29]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.00,-0.7393,-90.3518,Castillo Galapagos
6,Rikitea,PF,78.22,-23.1203,-134.9692,Pension Maro'i
21,Hithadhoo,MV,81.90,-0.6000,73.0833,Scoop Guest House
23,Saint-Joseph,RE,79.00,-21.3667,55.6167,"""Plantation Bed and Breakfast"
26,Buala,SB,86.09,-8.1450,159.5921,Maringe Lagoon Lodge
...,...,...,...,...,...,...
547,Kupang,ID,82.40,-10.1667,123.5833,Hotel On The Rock by Prasanthi
548,Novo Aripuana,BR,75.07,-5.1206,-60.3797,Hotel Ezequiel
549,Puerto Ayacucho,VE,75.27,5.6639,-67.6236,Posada Turistica Manapiare
560,Quatre Cocos,MU,82.00,-20.2078,57.7625,Emeraude Beach Attitude Hotel


In [32]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))